In [27]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [28]:
# TODO
# 選手名を特長量化
# 球種、球速、コースを予測するモデルを作成し、特長量化
# アルゴリズムの確認
# 特長量の有用性の確認

In [29]:
train = pd.read_csv('./input/train_data.csv')
test = pd.read_csv('./input/test_data.csv')

In [30]:
y_train = train['y']
X_train = train.drop(columns=['id', 'gameID', 'pitchType', 'speed', 'ballPositionLabel', 'ballX', 'ballY', 'dir', 'dist', 'battingType', 'isOuts', 'y'])
X_test = test.drop(columns=['id', 'gameID'])

In [31]:
# 0,1への変換
rep_dict = {False: 0, True: 1}
X_train.replace({'b1': rep_dict, 'b2': rep_dict, 'b3': rep_dict}, inplace=True)
X_test.replace({'b1': rep_dict, 'b2': rep_dict, 'b3': rep_dict}, inplace=True)

rep_dict = {'R': 0, 'L': 1}
X_train.replace({'pitcherHand': rep_dict, 'batterHand': rep_dict}, inplace=True)
X_test.replace({'pitcherHand': rep_dict, 'batterHand': rep_dict}, inplace=True)

In [32]:
# 選手名を数字に変換(train)
pitcher_dict = X_train['pitcher'].value_counts().to_dict()
batter_dict = X_train['batter'].value_counts().to_dict()

pitcher_id = {}
batter_id = {}

i = 0
for pitcher in pitcher_dict.keys():
    pitcher_id[pitcher] = i
    i += 1

i = 0
for batter in batter_dict.keys():
    batter_id[batter] = i
    i += 1

# testのみに存在する選手のidを-1に設定
for pitcher_test in X_test['pitcher'].value_counts().to_dict().keys():
    if not pitcher_test in pitcher_id:
        pitcher_id[pitcher_test] = -1

for batter_test in X_test['batter'].value_counts().to_dict().keys():
    if not batter_test in batter_id:
        batter_id[batter_test] = -1

X_train.replace({'pitcher': pitcher_id, 'batter': batter_id}, inplace=True)
X_test.replace({'pitcher': pitcher_id, 'batter': batter_id}, inplace=True)

In [33]:
# イニングと裏表を別のカラムにして裏表を0,1に変換
X_train.loc[X_train['inning'].str[-1] == '表', 'top_bottom'] = 0
X_train.loc[X_train['inning'].str[-1] == '裏', 'top_bottom'] = 1
X_train['inning'] = X_train['inning'].str[:-2]

X_test.loc[X_test['inning'].str[-1] == '表', 'top_bottom'] = 0
X_test.loc[X_test['inning'].str[-1] == '裏', 'top_bottom'] = 1
X_test['inning'] = X_test['inning'].str[:-2]

In [34]:
#　欠損値の埋め合わせ
X_train.fillna({'batterHand' :0, 'pitcherHand': 0, 'pitcher': -1, 'batter': -1}, inplace=True)
X_train[['pitcherHand', 'batterHand', 'inning', 'top_bottom', 'pitcher', 'batter']] = X_train[['pitcherHand', 'batterHand', 'inning', 'top_bottom', 'pitcher', 'batter']].astype(int)

X_test.fillna({'batterHand' :0, 'pitcherHand': 0, 'pitcher': -1, 'batter': -1}, inplace=True)
X_test[['pitcherHand', 'batterHand', 'inning', 'top_bottom', 'pitcher', 'batter']] = X_test[['pitcherHand', 'batterHand', 'inning', 'top_bottom', 'pitcher', 'batter']].astype(int)

In [35]:
X_test['pitcher'].dtype

dtype('int32')

In [36]:
# モデルの作成
clf = LogisticRegression()
clf.fit(X_train, y_train)

D:\anaconda3\envs\baseball_comp\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [37]:
y_pred = clf.predict(X_test)

In [38]:
y_test = test[['id']]
y_test['y'] = y_pred
y_pred = y_test

<ipython-input-38-8e283a71db6f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test['y'] = y_pred


In [39]:
y_pred.to_csv('output/y_pred.csv', index=False)
